# 데이터 전처리

In [1]:
import torch
import pandas as pd
import glob
import numpy as np
import re
import random
import tqdm
import math

c:\Users\seung\anaconda3\envs\DL\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 입력 데이터 생성

In [2]:
# 좌표 개수
point_n = 245

data_list = glob.glob("./original/*.txt")

# 수집 데이터 개수
sample_size = len(data_list)

x = [[] for _ in range(sample_size)]

# 동일 크기의 입력 데이터 생성
for i, path in enumerate(data_list):
    df = pd.read_table(path, sep=" ", dtype='f')
    x[i].append(pd.DataFrame(
        df, 
        columns=['X', 'Y', 'Z', 'F'], 
        index=np.linspace(0, len(df)-1, 
        point_n, 
        endpoint=True, 
        dtype='i')
    ))

# list to numpy
x = np.array(x, dtype='d').reshape(-1,4)

print(x.shape)

(6370, 4)


### 패턴화

In [3]:
pattern_n = 5    # 패턴 구간

# 패턴화 후 저장될 입출력 데이터
px = [[] for _ in range(sample_size)]
py = [] 

# 0~4 -> 0 / 1~5 -> 1 / 2~6 -> 2 / ...
for i in range(sample_size):
    for j in range(point_n - pattern_n):    # 슬라이딩 횟수
        px[i].append(x[i*point_n+j:i*point_n+j+pattern_n])
        py.append(j)

# list to numpy
px = np.array(px).reshape(-1,pattern_n,4)
py = np.array(py)

print(px.shape, py.shape)

(6240, 5, 4) (6240,)


### 데이터셋 저장

In [4]:
with open('./train/inputs.txt', 'w') as f:
    f.writelines('X,Y,Z,F' + '\n')
    for i in px:
        np.savetxt(f, i, delimiter=',', fmt='%.4f')

with open('./train/outputs.txt', 'w') as f:
    f.writelines('class' + '\n')
    np.savetxt(f, py, fmt='%d')